# Audio Injection with Magenta RT!

<a href="https://colab.research.google.com/github/magenta/magenta-realtime/blob/main/notebooks/Magenta_RT_Audio_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how live input audio can be “injected” into Magenta RT’s context to steer its generation. The model may repeat your input, or may *transform* it into something else! Try different prompts for different results.

### Tips for live mic input:

1. Use **wired** headphones! 🎧〰〰💻
1. **Rerun** ⏱️ `Measure latency` after any system audio change, like opening or closing an app.
1. Start by inputting something with a **clear rhythm**. You can tap along with the metronome, or count 1-2-3-4, or try other sounds!

### Known issues

- The model may drift off of the set tempo.
- Latency measurement may not work properly when routing audio from another application.

# Step 1: 😴 One-time setup

In [ ]:
# @title Run this cell to install dependencies (~5 minutes)
# @markdown Make sure you are running on **`v2-8 TPU` runtime** via `Runtime > Change Runtime Type`

# @markdown Colab may prompt you to restart session. **Wait until the cell finishes running to restart**!

# Clone library
!git clone https://github.com/magenta/magenta-realtime.git

# Magenta RT requires nightly TF builds, but stable may be installed.
# Force nightly to take precedence by uninstalling and reinstalling.
# Temporary workaround until MusicCoCa supported by TF stable.
_all_tf = 'tensorflow tf-nightly tensorflow-cpu tf-nightly-cpu tensorflow-tpu tf-nightly-tpu tensorflow-hub tf-hub-nightly tensorflow-text tensorflow-text-nightly'
_nightly_tf = 'tf-nightly==2.20.0.dev20250619 tensorflow-text-nightly==2.20.0.dev20250316 tf-hub-nightly'

# Install library and dependencies
# If running on TPU (recommended, runs on free tier Colab TPUs):
!pip install -e magenta-realtime/[tpu] && pip uninstall -y {_all_tf} && pip install {_nightly_tf}
# Uncomment if running on GPU (requires A100 via Colab Pro):
# !pip install -e magenta-realtime/[gpu] && pip uninstall -y {_all_tf} && pip install {_nightly_tf}

# Download the example of pre-recorded music
GS_EXAMPLES_DIR = 'gs://magenta-rt-public/colab_examples'
!gsutil cp -R $GS_EXAMPLES_DIR /content/ &> /dev/null

In [ ]:
# @title Run this cell to initialize model (~5 minutes)

import os

import concurrent.futures
import dataclasses
import functools
from google.colab import files
import ipywidgets as ipw
import IPython.display as ipd
import jax
import librosa
import numpy as np
import tempfile
from typing import Optional, Sequence, Tuple
import warnings

from magenta_rt import audio as audio_lib
from magenta_rt import musiccoca
from magenta_rt import spectrostream
from magenta_rt import system
from magenta_rt import utils
from magenta_rt.colab import prompt_types
from magenta_rt.colab import utils as colab_utils
from magenta_rt.colab import widgets


# ========= Helper functions ===========

def load_audio(audio_filename, sample_rate):
  """Loads an audio file.

  Args:
    audio_filename: File path to load.
    sample_rate: The number of samples per second at which the audio will be
        returned. Resampling will be performed if necessary.

  Returns:
    A numpy array of audio samples, sampled at the specified rate, in float32
    format.
  """
  y, unused_sr = librosa.load(audio_filename, sr=sample_rate, mono=False)
  return y


def wav_data_to_samples_librosa(audio_file, sample_rate):
  """Loads an in-memory audio file with librosa.

  Use this instead of wav_data_to_samples if the wav is 24-bit, as that's
  incompatible with wav_data_to_samples internal scipy call.

  Will copy to a local temp file before loading so that librosa can read a file
  path. Librosa does not currently read in-memory files.

  It will be treated as a .wav file.

  Args:
    audio_file: Wav file to load.
    sample_rate: The number of samples per second at which the audio will be
        returned. Resampling will be performed if necessary.

  Returns:
    A numpy array of audio samples, single-channel (mono) and sampled at the
    specified rate, in float32 format.
  """
  with tempfile.NamedTemporaryFile(suffix='.wav') as wav_input_file:
    wav_input_file.write(audio_file)
    # Before copying the file, flush any contents
    wav_input_file.flush()
    # And back the file position to top (not need for Copy but for certainty)
    wav_input_file.seek(0)
    return load_audio(wav_input_file.name, sample_rate)


def get_metronome_audio(
    loop_samples: int,
    beats_per_loop: int,
    sample_rate: int,
    chunk_samples: int):
  """Generates metronome audio.

  Args:
    loop_samples: The number of samples in a loop.
    beats_per_loop: The number of beats in a loop.
    sample_rate: The sample rate of the audio.
    chunk_samples: The number of samples in a chunk.

  Returns:
    A numpy array of metronome audio samples.
  """
  metronome_audio = np.zeros((loop_samples,))
  BEEP_SECONDS = 0.04
  BEEP_VOLUME = 0.25
  beeps = []
  for freq in (880, 440):
    beeps.append(BEEP_VOLUME * np.sin(np.linspace(
        0,
        2 * np.pi * freq * BEEP_SECONDS,
        int(BEEP_SECONDS * sample_rate))))
  ramp_samples = 100
  beep_envelope = np.concat(
      [np.linspace(0, 1, ramp_samples),
       np.ones((int(BEEP_SECONDS * sample_rate) - 2 * ramp_samples,)),
       np.linspace(1, 0, ramp_samples)])
  for i in range(len(beeps)):
    beeps[i] *= beep_envelope
  beat_length = loop_samples // beats_per_loop
  for i in range(beats_per_loop):
    beep = beeps[0 if i == 0 else 1]
    metronome_audio[i * beat_length:i * beat_length + len(beep)] = beep
  # Add an extra buffer to the metronome audio to make slicing easier later.
  return np.concat([metronome_audio, metronome_audio[:chunk_samples]])


# ================ Model System ======================
class MagentaRTCFGTied(system.MagentaRTT5X):
  """Magenta RT T5X system with "tied" CFG controlling input and style."""

  # This method is mostly identical to system.MagentaRTT5X.generate_chunk, but
  # adds "tied" CFG that acts jointly on input and style. Negative input tokens
  # are passed as `context_tokens_orig`.
  def generate_chunk(
      self,
      state: Optional[system.MagentaRTState] = None,
      style: Optional[musiccoca.StyleEmbedding] = None,
      seed: Optional[int] = None,
      **kwargs,
  ) -> Tuple[audio_lib.Waveform, system.MagentaRTState]:
    """Generates a chunk of audio and returns updated state.

    Args:
      state: The current state of the system.
      style: The style embedding to use for the generation.
      seed: The seed to use for the generation.
      **kwargs: Additional keyword arguments for sampling params, e.g.
        temperature, topk, guidance_weight, max_decode_frames.

    Returns:
      A tuple of the generated audio and the updated state.
    """
    # Init state, style, and seed (if not provided)
    if state is None:
      state = self.init_state()
    if seed is None:
      seed = np.random.randint(0, 2**31)

    context_tokens = {
        "orig": kwargs.get("context_tokens_orig", state.context_tokens),
        "mix": state.context_tokens,
    }
    codec_tokens_lm = {}
    for key, tokens in context_tokens.items():
      # Prepare codec tokens for LLM
      codec_tokens_lm[key] = np.where(
          tokens >= 0,
          utils.rvq_to_llm(
              np.maximum(tokens, 0),
              self.config.codec_rvq_codebook_size,
              self.config.vocab_codec_offset,
          ),
          np.full_like(tokens, self.config.vocab_mask_token),
      )
      assert (
          codec_tokens_lm[key].shape == self.config.context_tokens_shape
      )  # (250, 16)
      assert (
          codec_tokens_lm[key].min() >= self.config.vocab_mask_token
          and codec_tokens_lm[key].max()
          < (self.config.vocab_codec_offset + self.config.vocab_codec_size)
      )  # check range [1, 16386)

    # Prepare style tokens for LLM
    if style is None:
      style_tokens_lm = np.full(
          (self.config.encoder_style_rvq_depth,),
          self.config.vocab_mask_token,
          dtype=np.int32,
      )
    else:
      if style.shape != (self.style_model.config.embedding_dim,):
        raise ValueError(f"Invalid style shape: {style.shape}")
      style_tokens = self.style_model.tokenize(style)
      assert style_tokens.shape == (self.style_model.config.rvq_depth,)
      style_tokens = style_tokens[: self.config.encoder_style_rvq_depth]
      style_tokens_lm = utils.rvq_to_llm(
          style_tokens,
          self.config.style_rvq_codebook_size,
          self.config.vocab_style_offset,
      )
      assert (
          style_tokens_lm.min() >= self.config.vocab_style_offset
          and style_tokens_lm.max()
          < (self.config.vocab_style_offset + self.config.vocab_style_size)
      )  # check range [17140, 23554)
    assert style_tokens_lm.shape == (
        self.config.encoder_style_rvq_depth,
    )  # (6,)

    # Prepare encoder input
    batch_size, _, _ = self._device_params
    encoder_inputs_pos = np.concatenate(
        [codec_tokens_lm["mix"][
            :, :self.config.encoder_codec_rvq_depth].reshape(-1),
         style_tokens_lm
        ],
        axis=0,
    )
    assert encoder_inputs_pos.shape == (1006,)

    # Construct negative using original context tokens, and masking style.
    encoder_inputs_neg = np.concatenate(
        [codec_tokens_lm["orig"][
            :, :self.config.encoder_codec_rvq_depth].reshape(-1),
         style_tokens_lm
        ],
        axis=0,
    )
    encoder_inputs_neg[-self.config.encoder_style_rvq_depth:] = (
        self.config.vocab_mask_token)
    assert encoder_inputs_neg.shape == (1006,)

    encoder_inputs = np.stack([encoder_inputs_pos, encoder_inputs_neg], axis=0)
    assert encoder_inputs.shape == (2, 1006)

    # Generate tokens / NLL scores.
    max_decode_frames = kwargs.get(
        "max_decode_frames", self.config.chunk_length_frames
    )
    generated_tokens, _ = self._llm(
        {
            "encoder_input_tokens": encoder_inputs,
            "decoder_input_tokens": np.zeros(
                (
                    batch_size,
                    self.config.chunk_length_frames
                    * self.config.decoder_codec_rvq_depth,
                ),
                dtype=np.int32,
            ),
        },
        {
            "max_decode_steps": np.array(
                max_decode_frames * self.config.decoder_codec_rvq_depth,
                dtype=np.int32,
            ),
            "guidance_weight": kwargs.get(
                "guidance_weight", self._guidance_weight
            ),
            "temperature": kwargs.get("temperature", self._temperature),
            "topk": kwargs.get("topk", self._topk),
        },
        jax.random.PRNGKey(seed + state.chunk_index),
    )

    # Process generated tokens
    generated_tokens = np.array(generated_tokens)
    assert generated_tokens.shape == (
        batch_size,
        self.config.chunk_length_frames * self.config.decoder_codec_rvq_depth,
    )
    generated_tokens = generated_tokens[:1]  # larger batch sizes unsupported
    generated_tokens = generated_tokens.reshape(
        self.config.chunk_length_frames, self.config.decoder_codec_rvq_depth
    )  # (50, 16)
    generated_tokens = generated_tokens[:max_decode_frames]  # (N, 16)
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      generated_rvq_tokens = utils.llm_to_rvq(
          generated_tokens,
          self.config.codec_rvq_codebook_size,
          self.config.vocab_codec_offset,
          safe=False,
      )

    # Decode via SpectroStream using additional frame of samples for crossfading
    # We want to generate a 2s chunk with an additional 40ms of crossfade, which
    # is one additional codec frame. Caller is responsible for actually applying
    # the crossfade.
    xfade_frames = state.context_tokens[-self.config.crossfade_length_frames :]
    if state.chunk_index == 0:
      # NOTE: This will create 40ms of gibberish at the beginning but it's OK.
      xfade_frames = np.zeros_like(xfade_frames)
    assert xfade_frames.min() >= 0
    xfade_tokens = np.concatenate([xfade_frames, generated_rvq_tokens], axis=0)
    assert xfade_tokens.shape == (
        self.config.crossfade_length_frames + max_decode_frames,
        self.config.decoder_codec_rvq_depth,
    )  # (N+1, 16)
    waveform = self.codec.decode(xfade_tokens)
    assert isinstance(waveform, audio_lib.Waveform)
    assert waveform.samples.shape == (
        self.config.crossfade_length_samples
        + max_decode_frames * self.config.frame_length_samples,
        self.num_channels,
    )  # ((N+1)*1920, 2)

    # Update state
    state.update(generated_rvq_tokens)

    return (waveform, state)


spectrostream_model = spectrostream.SpectroStreamJAX(lazy=False)

SAMPLE_RATE = 48000
CHUNK_SECONDS = 2.0
CHUNK_SAMPLES = int(CHUNK_SECONDS * SAMPLE_RATE)
INPUT_AUDIO = None

# Fetch checkpoints and initialize model (may take up to 5 minutes)
MRT = MagentaRTCFGTied(tag="large", device="tpu:v2-8", lazy=False)




# Step 2: 🎵 Stream music with audio injection! 🎤

In [ ]:
# @title Input settings

# @markdown #### Select input source: 💾 Pre-recorded or 🎤 Live Mic

input_source = 'Pre-recorded' # @param ['Live Mic', 'Pre-recorded']
use_prerecorded_input = (input_source == 'Pre-recorded')

# @markdown #### 💾 Choose pre-recorded input

prerecorded_audio = "Electronic Groove" # @param ["Electronic Groove", "Funk Guitar", "Upload your own"]

# @markdown #### 🎤 Live mic calibration
# @markdown * Use wired headphones
# @markdown * Take headphones off and bring close to mic
# @markdown * Click `start` and wait ~10s for measurement


# === Calibration and Audio Upload ===
def load_example(fname):
    return load_audio(
        os.path.join('/content/colab_examples', fname),
        sample_rate=SAMPLE_RATE,
    )


if use_prerecorded_input:
  print('Using Pre-recorded Input!')

  if prerecorded_audio == "Electronic Groove":
    audio_samples = load_example("antoines_groove.mp3")

  if prerecorded_audio == "Funk Guitar":
    audio_samples = load_example("jesses_funk_guitar.mp3")

  if prerecorded_audio == "Upload your own":
    # Upload audio file
    audio_data = list(files.upload().values())[0]
    audio_samples = wav_data_to_samples_librosa(
        audio_data,
        sample_rate=SAMPLE_RATE,
    )

  # Postprocess
  if audio_samples.ndim == 2:
    audio_samples = audio_samples.T
  else:
    audio_samples = np.tile(audio_samples[:, None], 2)

  print("First 10s of input audio:")
  ipd.display(ipd.Audio(audio_samples[:SAMPLE_RATE * 10].T, rate=SAMPLE_RATE))

  # Add one buffer of looped audio to make slicing easier later.
  INPUT_AUDIO = np.concat([audio_samples, audio_samples[:CHUNK_SAMPLES]])

else:
  print('Using Live Mic Input!')
  # Calibrate latency.
  latency_estimator = colab_utils.LatencyEstimator(
    rate=SAMPLE_RATE,
    buffer_size=int(SAMPLE_RATE * CHUNK_SECONDS),
    duration=1,
  )

In [ ]:
# @title Metronome and Loop Settings

# @markdown <ul>
# @markdown <li>🎤 <b>Live Mic</b>: You'll perform a few intro loops to a metronome.</li>
# @markdown <li>💾 <b>Pre-recorded Input</b>: The model will join after a few intro loops.</li>
# @markdown </ul>

# @markdown Beats per minute (BPM) used for metronome:
bpm = 120  # @param {type: "number"}

# @markdown Mic audio is delayed by one loop of this many beats:
beats_per_loop = 8  # @param {"type":"slider","min":1,"max":16,"step":1}

loop_seconds = beats_per_loop * 60 / bpm
loop_samples = int(loop_seconds * SAMPLE_RATE)

# @markdown How many loops before the model joins in:

intro_loops = 4  # @param {"type":"slider","min":1,"max":5,"step":1}


METRONOME_AUDIO = get_metronome_audio(loop_samples, beats_per_loop, SAMPLE_RATE, CHUNK_SAMPLES)

print(f'Model will join in after {intro_loops * loop_seconds:.2f} seconds')


# === Advanced settings ===

# Extra prefix frames to include in the input+output mix that will be encoded.
# These may be used to update the model context.
MIX_PREFIX_FRAMES = 16

# Frames to trim from the left of the input+output mix before updating the
# model context, to avoid edge artifacts.
LEFT_EDGE_FRAMES_TO_REMOVE = 8
assert LEFT_EDGE_FRAMES_TO_REMOVE <= MIX_PREFIX_FRAMES




In [ ]:
# @title ▶️ Run this cell to start demo (~1 minute first time)

if not use_prerecorded_input:
  assert latency_estimator.done, "It looks like you forgot to run step 2!"
  latency_samples = latency_estimator.get_latency()
  assert 0 < latency_samples - 2 * CHUNK_SAMPLES < 0.3 * SAMPLE_RATE, (
      "Bad latency measurement. Rerun step 2.")


class AudioFade:
  """Handles the cross fade between audio chunks.

  Args:
    chunk_size: Number of audio samples per predicted frame (current
      SpectroStream models produces 25Hz frames corresponding to 1920 audio
      samples at 48kHz)
    num_chunks: Number of audio chunks to fade between.
    stereo: Whether the predicted audio is stereo or mono.
  """

  def __init__(self, chunk_size: int, num_chunks: int, stereo: bool):
    fade_size = chunk_size * num_chunks
    self.fade_size = fade_size
    self.num_chunks = num_chunks

    self.previous_chunk = np.zeros(fade_size)
    self.ramp = np.sin(np.linspace(0, np.pi / 2, fade_size)) ** 2

    if stereo:
      self.previous_chunk = self.previous_chunk[:, np.newaxis]
      self.ramp = self.ramp[:, np.newaxis]

  def reset(self):
    self.previous_chunk = np.zeros_like(self.previous_chunk)

  def __call__(self, chunk: np.ndarray) -> np.ndarray:
    chunk[: self.fade_size] *= self.ramp
    chunk[: self.fade_size] += self.previous_chunk
    self.previous_chunk = chunk[-self.fade_size :] * np.flip(self.ramp)
    return chunk[: -self.fade_size]


@dataclasses.dataclass
class AudioInjectionState:
  """State management for Audio Injection."""
  # The most recent context window (10s) of audio tokens corresponding to the
  # model's predicted output. These are parallel to `state.context_tokens` but
  # that context has the input audio mixed in.
  context_tokens_orig: np.ndarray
  # Stores all audio input (mono for live input, stereo for prerecorded input)
  all_inputs: np.ndarray
  # Stores all audio output (stereo)
  all_outputs: np.ndarray
  # How many chunks of audio have been generated
  step: int


# TODO(nconstant): Consider factoring out methods shared with MagentaRTStreamer
# from Magenta_RT_Demo.ipynb
class AudioInjectionStreamer:
  """Audio streamer class for Magenta RT model with Audio Injection.

  This class holds a pretrained Magenta RT model, a cross fade state, a
  generation state, audio injection state, and an asynchronous executor to
  handle prompt embedding without interrupting the audio thread.

  Args:
    system: A MagentaRTBase instance.
  """

  def __init__(
      self,
      system: system.MagentaRTBase,
      sample_rate: int = 48000,
      num_channels: int = 2,
      buffer_size: int = 2 * 48000,
      extra_buffering: int = 0
  ):
    config = system.config
    self.system = system
    self.audio_streamer = None
    self.sample_rate = sample_rate
    self.num_channels = num_channels
    self.buffer_size = buffer_size
    self.extra_buffering = extra_buffering
    self.fade = AudioFade(
        chunk_size=int(config.codec_sample_rate * config.crossfade_length),
        num_chunks=1,
        stereo=True
    )
    self.state = None
    self.executor = concurrent.futures.ThreadPoolExecutor()
    context_seconds = config.context_length
    context_frames = int(context_seconds * config.codec_frame_rate)
    context_samples = int(context_seconds * SAMPLE_RATE)
    self.injection_state = AudioInjectionState(
        context_tokens_orig=np.zeros(
            (context_frames, config.decoder_codec_rvq_depth),
            dtype=np.int32
        ),
        all_inputs=np.zeros(
            (context_samples, 2)
            if use_prerecorded_input else (context_samples,),
            dtype=np.float32
        ),
        all_outputs=np.zeros((context_samples, 2), dtype=np.float32),
        step=-1,  # This will be 0 after the warmup call.
    )

  @property
  def warmup(self):
    """Returns whether to warm up the audio streamer."""
    return True

  def on_stream_start(self):
    """Called when the UI starts streaming."""
    self.get_style_embedding(force_wait=False)
    self.get_style_embedding(force_wait=True)
    if self.audio_streamer is not None:
      self.audio_streamer.reset_ring_buffer()

  def on_stream_stop(self):
    """Called when the UI stops streaming."""
    pass

  def reset(self):
    self.state = None
    self.fade.reset()
    self.embed_style.cache_clear()
    if self.audio_streamer is not None:
      self.audio_streamer.reset_ring_buffer()

  def start(self):
    self.audio_streamer = colab_utils.AudioStreamer(
        self,
        rate=self.sample_rate,
        buffer_size=self.buffer_size,
        enable_input=True,
        warmup=self.warmup,
        raw_input_audio=True,
        enable_automatic_gain_control_on_input=True,
        num_output_channels=self.num_channels,
        additional_buffered_samples=self.extra_buffering,
        start_streaming_callback=self.on_stream_start,
        stop_streaming_callback=self.on_stream_stop,
    )
    self.reset()

  def stop(self):
    self.executor.shutdown(wait=True)
    if self.audio_streamer is not None:
      del self.audio_streamer
      self.audio_streamer = None

  def global_ui_params(self):
    return colab_utils.Parameters.get_values()

  @functools.cache
  def embed_style(self, style: str):
    return self.executor.submit(self.system.embed_style, style)

  @functools.cache
  def embed_16k_audio(self, audio: tuple[float]):
    """Embed 16k audio asyncronously, returning a future."""
    audio = audio_lib.Waveform(np.asarray(audio), 16000)
    return self.executor.submit(self.system.embed_style, audio)

  def embed_48k_audio(self, audio: tuple[float]):
    """Embed 48k audio asyncronously, returning an embedding."""
    resampled_audio = resampy.resample(np.asarray(audio), 48000, 16000)
    return self.embed_16k_audio(tuple(resampled_audio)).result()

  def get_style_embedding(self, force_wait: bool = False):
    prompts = self.get_prompts()
    weighted_embedding = np.zeros((768,), dtype=np.float32)
    total_weight = 0.0
    for prompt_value, prompt_weight in prompts:
      match type(prompt_value):
        case prompt_types.TextPrompt:
          if not prompt_value:
            continue
          embedding = self.embed_style(prompt_value)

        case prompt_types.AudioPrompt:
          embedding = self.embed_16k_audio(tuple(prompt_value.value))

        case prompt_types.EmbeddingPrompt:
          embedding = prompt_value.value

        case _:
          raise ValueError(f"Unsupported prompt type: {type(prompt_value)}")

      if isinstance(embedding, concurrent.futures.Future):
        if force_wait:
          embedding.result()

        if not embedding.done():
          continue

        embedding = embedding.result()

      weighted_embedding += embedding * prompt_weight
      total_weight += prompt_weight

    if total_weight > 0:
      weighted_embedding /= total_weight

    return weighted_embedding

  def get_prompts(self):
    params = self.global_ui_params()
    num_prompts = sum(map(lambda s: "prompt_value" in s, params.keys()))
    prompts = []
    for i in range(num_prompts):
      prompt_weight = params[f"prompt_weight_{i}"]
      prompt_value = params[f"prompt_value_{i}"]

      if prompt_value is None or not prompt_weight:
        continue

      match type(prompt_value):
        case prompt_types.TextPrompt:
          prompt_value = prompt_value.strip()
        case prompt_types.AudioPrompt:
          pass
        case prompt_types.EmbeddingPrompt:
          pass
        case _:
          raise ValueError(f"Unsupported prompt type: {type(prompt_value)}")

      prompts.append((prompt_value, prompt_weight))
    return prompts

  def generate(self, ui_params, inputs):
    if use_prerecorded_input:
      assert INPUT_AUDIO is not None, (
          "To use prerecorded input, first upload audio using step 4.")
      start = (self.injection_state.step * CHUNK_SAMPLES) % (
          len(INPUT_AUDIO) - CHUNK_SAMPLES)
      end = start + CHUNK_SAMPLES
      inputs = INPUT_AUDIO[start:end]
      inputs_mono = np.mean(inputs, axis=-1)

    if LIVE_AUDIO_PROMPT is not None:
      # Update live audio prompt with latest inputs.
      LIVE_AUDIO_PROMPT.update_audio_input(
          inputs_mono if use_prerecorded_input else inputs)

    # Add this input chunk to the end of `all_inputs`.
    self.injection_state.all_inputs = np.concatenate(
        [self.injection_state.all_inputs, inputs], axis=0
    )

    # Pass an extra prefix of mixed audio to the encoder so we can throw away
    # the earliest frames, which have edge artifacts.
    mix_samples = (
        CHUNK_SAMPLES + MIX_PREFIX_FRAMES
        * self.system.config.frame_length_samples
    )

    # Input audio will be delayed by one loop before being mixed with model
    # output.
    beats_per_loop = ui_params["beats_per_loop"]
    loop_seconds = beats_per_loop * 60 / bpm
    loop_samples = int(loop_seconds * SAMPLE_RATE)

    # "I/O offset" is the number of samples by which we shift the inputs
    # before mixing them with the outputs.
    io_offset = CHUNK_SAMPLES - int(
        streamer.system.config.crossfade_length * SAMPLE_RATE)
    if not use_prerecorded_input:
      io_offset += loop_samples - latency_samples
      # TODO(nconstant): Support "free" mode by padding with silence instead of
      # failing here.
      assert io_offset >= 0, (
          "Increase `beats_per_loop` in the previous cell and rerun it.")

    # Select a window of input audio for mixing.
    inputs_to_mix = self.injection_state.all_inputs[
        -(io_offset + mix_samples):-io_offset]

    # Select a window of output audio for mixing.
    outputs_to_mix = self.injection_state.all_outputs[-mix_samples:]
    outputs_to_mix *= ui_params.get("model_feedback")

    # Silence the last `input_gap_ms` ms of `inputs_to_mix`, to discourage
    # copying the input verbatim.
    input_gap_samples = int(SAMPLE_RATE * ui_params.get("input_gap") / 1000)
    ramp_samples = 100
    ramp = np.linspace(1, 0, min(ramp_samples, input_gap_samples))
    if use_prerecorded_input:
      ramp = np.stack([ramp, ramp], axis=-1)
    envelope = np.concat(
        [np.ones_like(inputs_to_mix[input_gap_samples:]),
         ramp,
         np.zeros_like(inputs_to_mix[:max(0, input_gap_samples - ramp_samples)])
        ]
    )
    inputs_to_mix = inputs_to_mix * envelope

    # Mix input and output audio.
    if not use_prerecorded_input:
      inputs_to_mix = inputs_to_mix[:, None]
    mix_audio = audio_lib.Waveform(
        inputs_to_mix + outputs_to_mix,
        sample_rate=SAMPLE_RATE
    )
    # Encode mix audio to tokens, and throw away a prefix.
    mix_tokens = spectrostream_model.encode(mix_audio)[
        LEFT_EDGE_FRAMES_TO_REMOVE:]

    if self.state is not None:
      self.injection_state.context_tokens_orig = self.state.context_tokens
      self.state.context_tokens[-len(mix_tokens):] = mix_tokens[
          :, :self.system.config.decoder_codec_rvq_depth]

    max_decode_frames = round(
        CHUNK_SECONDS * self.system.config.codec_frame_rate)

    chunk, self.state = self.system.generate_chunk(
        state=self.state,
        style=self.get_style_embedding(),
        seed=None,
        max_decode_frames=max_decode_frames,
        context_tokens_orig=self.injection_state.context_tokens_orig,
        **ui_params,
    )

    # Add this chunk (before cross-fading) to the end of `all_outputs`.
    # Note, we ignore the first frame of the chunk, which will be used for
    # cross-fading.
    self.injection_state.all_outputs = np.concatenate(
        [self.injection_state.all_outputs,
         chunk.samples[self.fade.fade_size:]]
    )

    chunk = self.fade(chunk.samples)
    chunk *= ui_params.get("model_volume")

    if ui_params.get("metronome"):
      # Add metronome audio to output.
      start = (self.injection_state.step * CHUNK_SAMPLES) % loop_samples
      end = start + CHUNK_SAMPLES
      metronome_chunk = METRONOME_AUDIO[start:end]
      chunk += metronome_chunk[:, None]

    if use_prerecorded_input:
      chunk += inputs * ui_params.get("input_volume")

    # When intro loops are over, raise model volume and feedback.
    if self.injection_state.step + 1 == int(
        intro_loops * loop_samples / CHUNK_SAMPLES):
      colab_utils.Parameters._UI_ELEMENTS["model_feedback"].value = 0.95
      colab_utils.Parameters._UI_ELEMENTS["model_volume"].value = (
          0.6 if use_prerecorded_input else 0.95)
      if not use_prerecorded_input:
         # Turn off metronome, and raise input gap.
        colab_utils.Parameters._UI_ELEMENTS["metronome"].value = False
        colab_utils.Parameters._UI_ELEMENTS["input_gap"].value = 400

    self.injection_state.step += 1

    return chunk

  def __call__(self, inputs):
    return self.generate(self.global_ui_params(), inputs)


streamer = AudioInjectionStreamer(MRT, buffer_size=CHUNK_SAMPLES)


def build_prompt_ui(
    default_prompts: Sequence[str],
    num_audio_prompt: int,
    include_live_audio_prompt: bool = False):
  """Add interactive prompt widgets and register them."""
  prompts = []

  if include_live_audio_prompt:
    live_audio_prompt = widgets.LiveAudioPrompt(
        streamer.embed_48k_audio,
        sample_rate=48000,
        trigger_embedding_every_n_seconds=0)
    live_audio_prompt.slider.value = 0.1
    prompts.append(live_audio_prompt)
    prompts[-1].slider.value = 0.1
  else:
    live_audio_prompt = None

  for p in default_prompts:
    prompts.append(widgets.Prompt())
    prompts[-1].text.value = p
  prompts[-len(default_prompts)].slider.value = 1.0

  # Add audio prompts
  for _ in range(num_audio_prompt):
    prompts.append(widgets.AudioPrompt())
    prompts[-1].slider.value = 0.0

  colab_utils.Parameters.register_ui_elements(
      display=False,
      **{f"prompt_weight_{i}": p.slider for i, p in enumerate(prompts)},
      **{f"prompt_value_{i}": p.prompt_value for i, p in enumerate(prompts)},
  )
  return live_audio_prompt, [p.get_widget() for p in prompts]


def build_sampling_option_ui():
  """Add interactive sampling option widgets and register them."""
  options = {
      "temperature": ipw.FloatSlider(
          min=0.0,
          max=4.0,
          step=0.01,
          value=1.2,
          description="temperature",
      ),
      "topk": ipw.IntSlider(
          min=0,
          max=1024,
          step=1,
          value=30,
          description="topk",
      ),
      "guidance_weight": ipw.FloatSlider(
          min=0.0,
          max=10.0,
          step=0.01,
          value=1.5 if use_prerecorded_input else 0.8,
          description="guidance",
      ),
  }
  if use_prerecorded_input:
    options.update({
        "model_volume": ipw.FloatSlider(
            min=0.0,
            max=1.0,
            step=0.05,
            value=0.0,
            description="model vol.",
        ),
        "input_volume": ipw.FloatSlider(
            value=1.0,
            min=0.0,
            max=1.0,
            step=0.05,
            orientation="horizontal",
            description="input volume",
        ),
    })
  else:
    options.update({
        "metronome": ipw.Checkbox(
            value=True,
            description="metronome",
            indent=False,
        ),
    })

  colab_utils.Parameters.register_ui_elements(display=False, **options)

  return list(options.values())


def build_hidden_option_ui():
  """Add interactive hidden option widgets and register them."""
  options = {
      "input_gap": ipw.IntSlider(
          value=0,
          min=0,
          max=2000,
          step=100,
          orientation="horizontal",
          description="input gap",
      ),
      "beats_per_loop": ipw.IntSlider(
          min=0,
          max=16,
          step=1,
          value=beats_per_loop,
          description="beats/loop",
      ),
      "model_feedback": ipw.FloatSlider(
          min=0.0,
          max=1.0,
          step=0.05,
          value=0.0,
          description="model feedback",
      ),
  }
  if not use_prerecorded_input:
    options.update({
        "model_volume": ipw.FloatSlider(
            min=0.0,
            max=1.0,
            step=0.05,
            value=0.0,
            description="model volume",
        ),
    })

  colab_utils.Parameters.register_ui_elements(display=False, **options)

  return list(options.values())


colab_utils.Parameters.reset()


def _reset_state(*args, **kwargs):
  del args, kwargs
  streamer.reset()


reset_button = ipw.Button(description="reset")
reset_button.on_click(_reset_state)

# Hidden injection sliders.
build_hidden_option_ui()

LIVE_AUDIO_PROMPT, prompts = build_prompt_ui(
    ["lofi hip hop beat",
     "funk jam",
     "acid house",
    ],
    num_audio_prompt=1,
)

# Building interactive UI
ipd.display(
    ipw.VBox([
        widgets.area(
            "sampling options",
            *build_sampling_option_ui(),
            reset_button,
        ),
        widgets.area(
            "prompts",
            *prompts,
        ),
    ])
)

streamer.start()

# Step 3: ⬇️ Download your session

Run the cells below to replay or download your input and the model output (as left and right channels).

In [ ]:
# @title Align saved input and output

context_seconds = streamer.system.config.context_length
context_samples = int(
    context_seconds * streamer.system.config.codec_sample_rate)

all_inputs = streamer.injection_state.all_inputs[context_samples:]
all_outputs = streamer.injection_state.all_outputs[context_samples:]

delay_samples = int(streamer.system.config.crossfade_length * SAMPLE_RATE)
if not use_prerecorded_input:
  delay_samples += latency_samples
delayed_inputs = np.concat(
    [all_inputs[delay_samples:],
     np.zeros_like(all_inputs[:delay_samples])
    ])

In [ ]:
# @title Input audio
print("\n")
if use_prerecorded_input:
  ipd.display(ipd.Audio(delayed_inputs.T, rate=SAMPLE_RATE))
else:
  ipd.display(ipd.Audio(delayed_inputs, rate=SAMPLE_RATE))

In [ ]:
# @title Output left
print("\n")
ipd.display(ipd.Audio(all_outputs[:, 0].T, rate=SAMPLE_RATE))

In [ ]:
# @title Output right
print("\n")
ipd.display(ipd.Audio(all_outputs[:, 1].T, rate=SAMPLE_RATE))

# License and terms

Magenta RealTime is offered under a combination of licenses: the codebase is
licensed under
[Apache 2.0](https://github.com/magenta/magenta-realtime/blob/main/LICENSE), and
the model weights under
[Creative Commons Attribution 4.0 International](https://creativecommons.org/licenses/by/4.0/legalcode).

In addition, we specify the following usage terms:

Copyright 2025 Google LLC

Use these materials responsibly and do not generate content, including outputs,
that infringe or violate the rights of others, including rights in copyrighted
content.

Google claims no rights in outputs you generate using Magenta RealTime. You and
your users are solely responsible for outputs and their subsequent uses.

Unless required by applicable law or agreed to in writing, all software and
materials distributed here under the Apache 2.0 or CC-BY licenses are
distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the licenses for the specific language governing
permissions and limitations under those licenses. You are solely responsible for
determining the appropriateness of using, reproducing, modifying, performing,
displaying or distributing the software and materials, and any outputs, and
assume any and all risks associated with your use or distribution of any of the
software and materials, and any outputs, and your exercise of rights and
permissions under the licenses.